<a href="https://colab.research.google.com/github/singh0021/TCD-Kaggle-Projects/blob/master/Kaggel_Linear_Regression_for_Income_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#I have built various model and tried various approaches. This notebook is specific to final solution. If anyone interested to know other approaches and visualization techniques. Please refer to other #notebook in same directory with name : Machine Learning Solution

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

# Lets load both training and Testing Data

In [0]:
train_df =pd.read_csv("Data.csv")
train_df = train_df.drop(['Instance'], axis=1)
test_df = pd.read_csv("tcdml2019-20incomepredictiontest(without labels).csv")
#train_df = train_df[(train_df['Income in EUR'] > 0) ]

# Lets describe the training data and we see that there are negative incomes and few outliers. We can get rid of those rows

In [0]:
# train_df.describe()
#train_df = train_df[(train_df['Income in EUR'] > 0)]
#train_df = train_df[train_df['Age'] < 110]
#train_df = train_df[train_df['Body Height [cm]'] < 225]

# Since there are 4-5 categorical variables and we have tested accucary of using few encoding such as one hot, label and target encoding. Target encoding gives me the best accurcay

In [0]:
def target_encoding(data, target, categ_variables, smooth):
    """    
  
    """
    train_target = data.copy()
    code_map = dict()   
    default_map = dict() 
    
    for v in categ_variables:
        prior = data[target].mean()
        n = data.groupby(v).size()
        mu = data.groupby(v)[target].mean()
        mu_smoothed = (n * mu + smooth * prior) / (n + smooth)
        
        train_target.loc[:, v] = train_target[v].map(mu_smoothed)        
        code_map[v] = mu_smoothed
        default_map[v] = prior        
    return train_target, code_map, default_map

In [39]:
len(train_df['Profession'].unique())


1341

In [0]:
categories = ['Country', 'Profession', 'University Degree','Gender']
train_smooth, target_m, d_m = target_encoding(train_df, 'Income in EUR', categories, 10)
test_t = test_df.copy()
for v in categories:
    test_t.loc[:, v] = test_t[v].map(target_m[v])

# We ran this model multiple times to see which features are adding value to the model. Hence we have selected below feature which gives best accuracy so far.

In [0]:
final_trhot = train_smooth[['Age','Year of Record','Body Height [cm]', 'Country', 'University Degree','Profession'
                                  ,'Size of City','Gender']]
test_col = test_t[['Age','Year of Record','Body Height [cm]', 'Country', 'University Degree','Profession',
                              'Size of City', 'Gender']]

In [9]:
# lets fill those missing values by taking means and since Income variable is skewed. We will take its log

final_trhot = final_trhot.fillna(final_trhot.mean())
test_col = test_col.fillna(test_col.mean())
y_t = np.log(train_df['Income in EUR'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log
  after removing the cwd from sys.path.


In [11]:
# Lets build linear model and check its accuracy

from sklearn.model_selection import train_test_split
Xhot_tr, Xhot_ts, Yhot_tr, Yhot_ts = train_test_split(final_trhot, y_t, test_size=0.25, random_state = 7)


from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

lm = linear_model.LinearRegression()


lm.fit(Xhot_tr,Yhot_tr)
y_pred = lm.predict(Xhot_ts)
print('The accuracy of the Linear Regression is',r2_score(Yhot_ts,y_pred))
print ('MSE is: \n', (mean_squared_error(Yhot_ts,y_pred)))


The accuracy of the Linear Regression is 0.6804993708026185
MSE is: 
 0.3844242452255641


In [0]:
#from sklearn.preprocessing import StandardScaler
#scaler = StandardScaler().fit(Xhot_tr)
#rescaled_X_train = scaler.transform(Xhot_tr)

#lm.fit(rescaled_X_train,Yhot_tr)

#rescaled_X_ts = scaler.transform(Xhot_ts)
#predictions = lm.predict(rescaled_X_ts)
#print('The accuracy of the Linear Regression is',r2_score(Yhot_ts,predictions))
#print ('MSE is: \n', np.sqrt(mean_squared_error(Yhot_ts,predictions)))


In [0]:
#import numpy
#from sklearn import linear_model
#from sklearn.ensemble import RandomForestRegressor

#cls = RandomForestRegressor(n_estimators=250, criterion='mse', max_depth=30)

#cls.fit(rescaled_X_train,Yhot_tr)
#y_pred = cls.predict(rescaled_X_ts)

#print('The accuracy of the Linear Regression is',r2_score(Yhot_ts,y_pred))
#print ('RMSE is: \n', np.sqrt(mean_squared_error(Yhot_ts,y_pred)))


In [12]:
# Lets build regression model and check its accuracy

import numpy
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor

cls = RandomForestRegressor(n_estimators=250, criterion='mse', max_depth=30)

cls.fit(Xhot_tr, Yhot_tr)
y_pred = cls.predict(Xhot_ts)

print('The accuracy of the Linear Regression is',r2_score(Yhot_ts,y_pred))
print ('RMSE is: \n', (mean_squared_error(Yhot_ts,y_pred)))


The accuracy of the Linear Regression is 0.9061450898599498
RMSE is: 
 0.11292654753744583


# It is also important to note that RMSE score decreases if we take log of Income variable on validation data. We shall stick to this approach only. Also, if we are not taking log of Income variable, we need to scale our data. Result are pretty much same on testing data only RMSE differs on local validation data

In [0]:
import re

regex = re.compile(r"\[|\]|<", re.IGNORECASE)

final_trhot.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in final_trhot.columns.values]
#Xhot_ts.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in Xhot_ts.columns.values]
test_col.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in test_col.columns.values]

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
Xhot_tr, Xhot_ts, Yhot_tr, Yhot_ts = train_test_split(final_trhot, train_df['Income in EUR'], test_size=0.25, random_state = 7)

scaler = StandardScaler().fit(Xhot_tr)
rescaled_X_train = scaler.transform(Xhot_tr)

rescaled_X_ts = scaler.transform(Xhot_ts)

my_model = XGBRegressor(n_estimators=1000)
xgb = my_model.fit(rescaled_X_train, Yhot_tr, early_stopping_rounds=5, 
             eval_set=[(rescaled_X_ts, Yhot_ts)], verbose=False)

y_pred = xgb.predict(rescaled_X_ts)

print('The accuracy of the XGB Regression is',r2_score(Yhot_ts,y_pred))
print ('RMSE is: \n', np.sqrt(mean_squared_error(Yhot_ts,y_pred)))


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[18:20:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The accuracy of the XGB Regression is 0.8388044065504278
RMSE is: 
 58988.46894758571


In [18]:
len(rescaled_X_ts)

73230

In [24]:
from sklearn import ensemble

clf = ensemble.GradientBoostingRegressor(n_estimators = 1200, max_depth = 7, min_samples_split = 2,
          learning_rate = 0.1, loss = 'ls')

clf.fit(rescaled_X_train, Yhot_tr)

rescaled_X_ts = scaler.transform(Xhot_ts)
predictions = clf.predict(rescaled_X_ts)

print('The accuracy of the XGB Regression is',r2_score(Yhot_ts,y_pred))
print (np.sqrt(mean_squared_error(Yhot_ts, predictions)))

The accuracy of the XGB Regression is 0.8388044065504278
58879.07945487714


In [49]:
test_col.columns

Index(['Age', 'Year of Record', 'Body Height _cm_', 'Country',
       'University Degree', 'Profession', 'Size of City', 'Gender'],
      dtype='object')

In [0]:
#rescaled_X_ts = scaler.transform(test_col)
#predictions = cls.predict(rescaled_X_ts)
#actual_predictions = predictions

# Since we have take log of Income variable when buidling model, now we need to inverse log so that we can get actual value of 
#incomes

#Lets do prediction on testing data

test_X_ts = scaler.transform(test_col)
#predictions = xgb.predict(test_col)
#predictions = xgb.predict(rescaled_X_ts)
predictions = clf.predict(test_X_ts)
#actual_predictions = np.exp(predictions)


In [26]:
predictions.max()

4277315.508772634

In [27]:
submissions =pd.DataFrame({'Instance': test_df['Instance'], 'Income': predictions}, columns =['Instance','Income'])
filename ='tcd ml 2019-20 income prediction submission file.csv'
submissions.to_csv(filename, index=False)
print('Saved File '+ filename)

Saved File tcd ml 2019-20 income prediction submission file.csv
